In [1]:
from typing import Literal

from dotenv import load_dotenv
from notebooks._utils import (
    celsius_to_fahrenheit,
    city_to_number,
    city_to_weather_condition,
)
from baml_agents import with_model
from baml_client.sync_client import b
import jupyter_black
import stackprinter

stackprinter.set_excepthook()
load_dotenv()
jupyter_black.load()

b = with_model(b, "gpt-4.1-nano")

## Creating tools

Personally, I prefer the term "Actions," so I'll use "Tools" and "Actions" interchangeably.

Let's create a tool for an AI to use (Action). We'll be using Pydantic-model-as-a-function pattern rather than regular typed Python functions because that makes it easer to add parameter validation, type hints, and baml-specific (LLM-specific) information to the function parameters.

So instead of writing...:

In [2]:
from baml_agents import Result


def get_weather_info(
    city: str,
    measurement: Literal["celsius", "fahrenheit"] = "celsius",
) -> Result:
    """Get weather information for a given city."""
    return Result(
        content="The weather in London is 18 degrees celsius with windy conditions.",
        error=False,
    )

...we will do:

In [3]:
from pydantic import Field
from baml_agents import Action


class GetWeatherInfo(Action):
    """Get weather information for a given city."""

    city: str
    measurement: Literal["celsius", "fahrenheit"] | None = None

    def run(self) -> Result:
        self.measurement = self.measurement or "celsius"
        c = city_to_number(self.city, -10, 35)
        condition = city_to_weather_condition(self.city)
        if self.measurement.lower() == "fahrenheit":
            c, u = celsius_to_fahrenheit(c), "fahrenheit"
        else:
            u = "celsius"
        content = f"The weather in {self.city} is {round(c, 1)} degrees {u} with {condition.lower()} conditions."
        return Result(content=content, error=False)

Let's try some cities:

In [4]:
print(GetWeatherInfo(city="New York", measurement="fahrenheit").run())
print(GetWeatherInfo(city="London").run())
print(GetWeatherInfo(city="Paris").run())

content='The weather in New York is 48.2 degrees fahrenheit with cloudy conditions.' error=False
content='The weather in London is -7 degrees celsius with cloudy conditions.' error=False
content='The weather in Paris is 34 degrees celsius with stormy conditions.' error=False


## Manually configuring baml

According to [12 Factor Agents](https://github.com/humanlayer/12-factor-agents), [tools are simply structured outputs](https://github.com/search?q=repo%3Ahumanlayer%2F12-factor-agents+Tools+are+just+structured+outputs&type=code).

That's why we'll just create a baml prompt template:

In [5]:
# This command shows file contents
!awk '/^class BamlCustomTools_GetWeatherInfo_Parameters/,/^}/' ../baml_src/notebooks/02_baml_custom_tools.baml
!awk '/^function BamlCustomTools_GetWeatherInfo/,/^}/' ../baml_src/notebooks/02_baml_custom_tools.baml

class IntroToTools_GetWeatherInfo_Parameters {
    city string
    measurement "celsius" | "fahrenheit" @description(#"default: celsius"#)
}
function IntroToTools_GetWeatherInfo(text: string) -> IntroToTools_GetWeatherInfo_Parameters {
  client Default
  prompt #"
    {{ text }}
    {{ ctx.output_format }}
  "#
}


Let's see the actual prompt that we will send to the LLM:

In [6]:
from baml_agents import display_prompt

question = "What's the weather like in the capital of Japan?"

request = b.request.BamlCustomTools_GetWeatherInfo(question)
display_prompt(request)

[system]
What's the weather like in the capital of Japan?
Answer in JSON using this schema:
{
  city: string,
  // default: celsius
  measurement: "celsius" or "fahrenheit",
}


Let's send it and run the result:

In [7]:
question = "What's the weather like in the capital of Japan?"

# Calling LLM
completion = b.BamlCustomTools_GetWeatherInfo(question)
print(completion)

city='Tokyo' measurement='celsius'


In [8]:
# Running the action
GetWeatherInfo.validate(completion).run()

Result(content='The weather in Tokyo is 20 degrees celsius with cloudy conditions.', error=False)

## Automatically configuring baml

We'll be using the [dynamic features of baml](https://docs.boundaryml.com/guide/baml-advanced/dynamic-runtime-types).

In [9]:
# This command shows file contents
!awk '/^class NextAction/,/^}/' ../baml_src/notebooks/02_baml_custom_tools.baml
!awk '/^function GetNextAction/,/^}/' ../baml_src/notebooks/02_baml_custom_tools.baml

class NextAction {
    @@dynamic
}
function GetNextAction(goal: string) -> NextAction {
    client Default
    prompt #"
        Please select the next best action to achieve the goal.
        
        <goal>
        {{ goal }}
        </goal>
        
        {{ ctx.output_format(prefix="Answer in JSON format with exactly one next best action:\n") }}
    "#
}


Let's see the actual prompt that we will send to the LLM:

In [10]:
from baml_agents import ActionRunner

question = "What's the weather like in the capital of Japan?"

r = ActionRunner()
r.add_action(GetWeatherInfo)
request = b.request.GetNextAction(question, baml_options={"tb": r.tb()})
display_prompt(request)

[system]
Please select the next best action to achieve the goal.

‹goal›
What's the weather like in the capital of Japan?
‹/goal›

Answer in JSON format with exactly one next best action:
{
  chosen_action: {
    // Get weather information for a given city.
    action_id: "get_weather_info",
    city: string,
    measurement: "celsius" or "fahrenheit" or null,
  },
}


Let's send it and run the result:

In [11]:
question = "What's the weather like in the capital of Japan?"

# Calling the LLM
r = ActionRunner()
r.add_action(GetWeatherInfo)
action = b.GetNextAction(question, baml_options={"tb": r.tb()})
print(action)

chosen_action={'action_id': 'get_weather_info', 'city': 'Tokyo', 'measurement': None}


In [12]:
r.run(action)

Result(content='The weather in Tokyo is 20 degrees celsius with cloudy conditions.', error=False)